In [1]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
def get_binary(n):
    binary_representation = []
    while n > 0:
        binary_representation.append((n & 1))
        n = n >> 1

    return binary_representation


def get_decimal(binary_representation):
    result = 0
    k = 0
    while len(binary_representation) > 0:
        digit = binary_representation.pop()
        result = result + digit * (2 ** k)
        k += 1
    
    return result

def pad(seq, length, padding=None):
    if padding == None:
        padding = 0
    z = [padding] * (length - len(seq))
    seq = seq + z
    
    return seq

In [11]:
class RNNModel(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(RNNModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers)
        self.fc = nn.Linear(hidden_dim, 2)
    
    def forward(self, x, hidden):
        output, hidden = self.rnn(x, hidden)
        output = self.fc(output)
        # output = F.softmax(output, dim=2)
        return output, hidden


class RNNModelEfficient(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(RNNModelEfficient, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers)
        self.fc = nn.Linear(hidden_dim, 3)
    
    def forward(self, x, lengths, hidden):
        sequence_length = x.size(0)
        batch_size = x.size(1)
        x = nn.utils.rnn.pack_padded_sequence(x, lengths)
        output, hidden = self.rnn(x, hidden)
        output, lengths_unpacked = nn.utils.rnn.pad_packed_sequence(output)
        output = self.fc(output)

        return output, hidden


class RNNModelFinal(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(RNNModelFinal, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers)
        self.fc = nn.Linear(hidden_dim, 3)
    
    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to("cuda")
    
    def forward(self, x, lengths=None):
        batch_size = x.size(1)
        hidden = self.init_hidden(batch_size)
        if lengths is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, lengths)
            output, hidden = self.rnn(x, hidden)
            output, lengths_unpacked = nn.utils.rnn.pad_packed_sequence(output)
        else:
            output, hidden = self.rnn(x, hidden)

        output = self.fc(output)

        return output

In [12]:
rnn = RNNModelFinal(2, 3, 1).cuda()
hidden = torch.zeros(1, 1, 3).to("cuda")
optimizer = optim.Adam(rnn.parameters(), lr=0.001)

In [5]:
dataset = []
for i in range(500):
    a = random.randrange(10000)
    for j in range(500):
        b = random.randrange(1000000)
        c = a + b
        x = get_binary(a)
        y = get_binary(b)
        z = get_binary(c) + [2]
        max_length = max(len(x), len(y), len(z))
        x = pad(x, max_length)
        y = pad(y, max_length)
        dataset.append((list(zip(x, y)), z))


In [6]:
batch_size = 100
batched_dataset = []
prev = 0
for i in range(batch_size, len(dataset), batch_size):
    batch = dataset[prev:i]
    batch = sorted(batch, key=lambda r: len(r[0]), reverse=True)
    lengths_batch = list(map(lambda r: len(r[0]), batch))
    max_length = max(lengths_batch)
    batch_padded = list(map(lambda s: pad(s[0], max_length, padding=(0, 0)), batch))
    batch_targets = list(map(lambda r: r[1], batch))
    batched_dataset.append(((batch_padded, lengths_batch), batch_targets))
    prev = i

In [7]:
training_set = batched_dataset[:int(0.8 * len(batched_dataset))]
testing_set = batched_dataset[int(0.8 * len(batched_dataset)):]

In [8]:
def batch_to_tensor(data_batch, lengths, batchsize):
    batch_tensors = list(map(lambda t: torch.tensor(t, dtype=torch.float32), data_batch))
    return torch.stack(batch_tensors, dim=1)

In [9]:
training_set_batches = []
testing_set_batches = []
for ((data_batch, batch_lengths), target) in training_set:
    training_set_batches.append(((batch_to_tensor(data_batch, batch_lengths, batch_size), batch_lengths), target))

for ((data_batch, batch_lengths), target) in testing_set:
    testing_set_batches.append(((batch_to_tensor(data_batch, batch_lengths, batch_size), batch_lengths), target))

In [13]:
for n in range(2):
    for ((data_batch, lengths_batch), target) in training_set_batches:
        hidden = torch.zeros(1, batch_size, 3).to("cuda")
        rnn.zero_grad()
        output = rnn(data_batch.to("cuda"), lengths=lengths_batch)
        loss = 0
        for i in range(batch_size):
            loss += torch.nn.functional.cross_entropy(output[:lengths_batch[i],i,:], torch.tensor(target[i]).to("cuda"))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [14]:
batch_id = 103
test_b, lengths_b = testing_set_batches[batch_id][0]
target_b = testing_set_batches[batch_id][1]

In [15]:
out = rnn(test_b.to("cuda"), lengths=lengths_b)
out.size()

torch.Size([21, 100, 3])

In [16]:
output = torch.argmax(out, dim=2)

In [17]:
output[:,2]

tensor([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 2, 1, 0],
       device='cuda:0')

In [18]:
torch.tensor(target_b[2])

tensor([0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 2])

In [43]:
x = torch.stack([torch.tensor([1, 1, 0, 0, 0, 0, 0], dtype=torch.float32), torch.tensor([1, 0, 1, 0, 0, 0, 0], dtype=torch.float32)], dim=0).view(7, 1, 2)

In [44]:
x

tensor([[[1., 1.]],

        [[0., 0.]],

        [[0., 0.]],

        [[0., 1.]],

        [[0., 1.]],

        [[0., 0.]],

        [[0., 0.]]])

In [45]:
oo = rnn(x.to("cuda"), lengths=[7])

In [46]:
ooo = torch.argmax(oo, dim=2)

In [47]:
ooo[:,0]

tensor([0, 1, 0, 1, 1, 2, 0], device='cuda:0')